# Cross-Resonance 门

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程介绍如何使用量脉生成 Cross-Resonance（CR）门的优化脉冲序列。与之前教程中的 iSWAP 和 CZ 门的实现不同，CR 门是使用全微波驱动实现的。本教程的概要如下：

- 背景介绍
- 准备工作
- 构造哈密顿量
- 通过量脉云服务生成与优化脉冲
- 总结

## 背景介绍

**基本原理**

与我们之前介绍的量子门不同的是，CR 门只使用微波来实现两个量子比特的相互作用，这样我们就可以避免由磁通而引起的噪声。CR 门的物理实现涉及两个耦合的频率固定的量子比特，并且使驱动脉冲的频率等于目标量子比特（Target Qubit）的频率，如下图所示：


![cr-circuit](figures/cr-circuit.png)


我们先关注系统的有效哈密顿量（详见参考文献 \[1\]）。在双旋转坐标系中，我们用驱动强度 $A$、失谐量 $\Delta$、驱动相位 $\phi_0$ 和耦合强度 $g_{01}$ 表示 cross-resonance 效应的有效哈密顿量（为了简洁，我们令 $\hbar=1$）：

$$
\hat{H}_{\rm eff} = \frac{A}{4\Delta}g_{01}(\hat{\sigma}_0^z\hat{\sigma}_1^x\cos{\phi_0}+\hat{\sigma}_0^z\hat{\sigma}_1^y\sin{\phi_0}).
$$

当 $\phi_0=0$ 时，cross-resonance 效应产生了 $\hat{\sigma}^z_0\otimes\hat{\sigma}_1^x$ 有效耦合。因此，我们可以从上面的有效哈密顿量推导出时间演化算符：

$$
U_{\rm CR}(\theta)=e^{-i\frac{\theta}{2}\hat{\sigma}^z_0\otimes\hat{\sigma}^x_1},
$$

其中 $\theta=\Omega_0 g_{01}t/(2\Delta)$（$t$ 是门时间）。可见，cross-resonance 效应使得量子比特 1（目标量子比特）的旋转取决于量子比特 0（控制量子比特）的状态。



通过上面的推导, CR 门的矩阵形式是（有关更多详细信息，请参阅 \[2\]）：
$$
U_{\rm CR}(\theta) = \begin{bmatrix} 
\cos{\frac{\theta}{2}} & -i\sin{\frac{\theta}{2}} & 0 & 0 \\
-i\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}} & 0 & 0 \\ 
0 & 0 & \cos{\frac{\theta}{2}} & i\sin{\frac{\theta}{2}} \\
0 & 0 & i\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}} 
\end{bmatrix}.
$$

 
特别的，当 $\theta=-\frac{\pi}{2}$ 时，CR 门的矩阵表示为：

$$
U_{\rm CR}(-\pi/2) = \frac{\sqrt{2}}{2} 
\begin{bmatrix}
1 & i & 0 & 0 \\
i & 1 & 0 & 0 \\
0 & 0 & 1 & -i \\
0 & 0 & -i & 1
\end{bmatrix}.
$$

**应用**

在分析了实现 CR 门的一些基本原理之后，我们现在关注 CR 门在量子计算中的应用：其中之一是通过增加两个额外的单量子比特门来实现 CNOT 门。

![cr-note](figures/cr-gate.png) 

在本教程中，我们对由两个三能级的量子比特组成的系统进行建模，并以目标量子比特（量子比特 $q_1$）的频率向控制量子比特（量子比特 $q_0$）施加驱动脉冲。通过旋转波近似（RWA），哈密顿量可以表示为（更多细节请参阅 \[1\]）：

$$
\hat{H}_{\rm sys} = (\omega_{\rm q0}-\omega_{\rm d})\hat{a}_{0}^{\dagger }\hat{a}_0 + (\omega_{\rm q1}-\omega_{\rm d})\hat{a}_1^\dagger \hat{a}_1 + \frac{\alpha_0}{2} \hat{a}^{\dagger2}_0\hat{a}^2_0 + \frac{\alpha_1}{2} \hat{a}^{\dagger2}_1\hat{a}^2_1+\frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger + \hat{a}_0^\dagger\hat{a}_1) + \Omega_0^x(t)\frac{\hat{a}^\dagger_0+\hat{a}_0}{2}.
$$

其中各种符号定义见下表：


|符号|定义|
|:--------:|:----------:|
|$\omega_{\rm qi}$| 量子比特 $q_i$ 的频率|
|$\omega_{\rm d}$|驱动频率|
|$\hat{a}_i^{\dagger}$|产生算符|
|$\hat{a}_i$|湮灭算符|
|$\alpha_i$| 量子比特 $q_i$ 的非谐性|
|$g$|耦合强度|
|$\Omega_0^x$(t)| X 通道的脉冲函数|

## 准备工作

成功安装量脉后，您可以按照本教程运行下面的量脉程序。要运行此教程，您需要从 Quanlse 和其它常用的 Python 库导入以下包：

In [ ]:
# Import Hamiltonian-related module
from Quanlse.QHamiltonian import QHamiltonian as QHam
from Quanlse.QOperator import driveX, number, duff

# Import optimizer for the cross-resonance gate
from Quanlse.remoteOptimizer import remoteOptimizeCr

# Import tools to analyze the result
from Quanlse.Utils.Functions import project
from Quanlse.Utils.Infidelity import unitaryInfidelity

# Import numpy and math
from numpy import round
from math import pi

## 构造哈密顿量


现在，我们需要使用量脉来构造哈密顿量。在量脉中，所有关于哈密顿量的信息都存储在一个字典中。我们首先定义一些构造哈密顿量的字典所需的基本参数：采样周期、系统中的量子比特数量和系统的能级。为了初始化该哈密顿量的字典，我们从模块 `QHamiltonian` 调用函数 `QHamiltonian()`。

In [ ]:
# Sampling period
dt = 1.0

# Number of qubits
qubits = 2

# System energy level
level = 3

# Initialize the Hamiltonian
ham = QHam(subSysNum=qubits, sysLevel=level, dt=dt)

现在我们可以开始构造哈密顿量了。在开始之前，我们需要定义几个常量作为函数的参数：

In [ ]:
# Parameters setting  
qubitArgs = {
    "coupling": 0.0038 * (2 * pi),  # Coupling of Q0 and Q1
    "qubit_freq0": 5.114 * (2 * pi),  # Frequency of Q0
    "qubit_freq1": 4.914 * (2 * pi),  # Frequency of Q1
    "drive_freq0": 4.914 * (2 * pi),  # Drive frequency on Q0
    "drive_freq1": 4.914 * (2 * pi),  # Drive frequency on Q1
    "qubit_anharm0": -0.33 * (2 * pi),  # Anharmonicity of Q0
    "qubit_anharm1": -0.33 * (2 * pi)  # Anharmonicity of Q1
}

然后我们需要在之前初始化的哈密顿的字典中添加以下项：

$$
\begin{align}
\hat{H}_{\rm drift} &= (\omega_{\rm q0}-\omega_{\rm d}) \hat{a}_0^\dagger \hat{a}_0 + (\omega_{\rm q1}-\omega_{\rm d}) \hat{a}_1^\dagger \hat{a}_1 + \frac{\alpha_0}{2} \hat{a}_0^{\dagger}\hat{a}_0^{\dagger}\hat{a}_0 
\hat{a}_0 + \frac{\alpha_1}{2} \hat{a}_1^{\dagger}\hat{a}_1^{\dagger}\hat{a}_1 \hat{a}_1 , \\
\hat{H}_{\rm coup} &= \frac{g_{01}}{2}(\hat{a}_0 \hat{a}_1^\dagger+\hat{a}^\dagger_0 \hat{a}_1). \\
\end{align}
$$

在量脉的 `Operator` 模块中，我们提供了一些允许用户快速构造常用算符的工具。失谐项 $(\omega_{\rm q}-\omega_{\rm d})\hat{a}^\dagger\hat{a}$ 和非谐项 $\frac{\alpha}{2}\hat{a}^\dagger\hat{a}^\dagger\hat{a}\hat{a}$ 可以分别使用 `Operator` 模块中的 `number(n)` 和 `duff(n)` 生成：这两个函数 `number(n)` 和 `duff(n)` 分别返回 $n\times n$ 的粒子数算符和 Duffing 算符。耦合项的形式为 $\frac{g}{2}(\hat{a}_i^\dagger\hat{a}_j+\hat{a}_i\hat{a}_j^\dagger)$，可以使用函数 `addCoupling()` 直接添加到哈密顿量中。

In [ ]:
# Add the detuning and the anharmonicity terms
for qu in range(2):
    # Add the detuning term(s).
    ham.addDrift(number(level), qu, (qubitArgs[f"qubit_freq{qu}"] - qubitArgs[f"drive_freq{qu}"]))
    # Add the anharmonicity term(s).
    ham.addDrift(duff(level), qu, qubitArgs[f"qubit_anharm{qu}"] / 2)

# Add the coupling term
ham.addCoupling([0, 1], qubitArgs["coupling"] / 2)

需要注意的是，量脉的优化函数会自动添加控制项：

$$
\hat{H}_{\rm ctrl}(t) = \Omega_0^x(t)\frac{\hat{a}^\dagger_0+\hat{a}_0}{2},
$$

所以我们不需要手动添加这项。

系统的哈密顿量构造完成后，我们可以进行量子系统的模拟。

## 通过量脉云服务生成与优化脉冲

在本地设备上处理优化过程通常需要很长时间，而我们提供的云服务可以显著加速此过程。要使用量脉云服务，用户需要从 http://quantum-hub.baidu.com 获取 token。

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerCr on cloud, paste your token (a string) here
Define.hubToken = ''

为了找到 CR 门的优化脉冲，我们使用函数 `remoteOptimizeCr()`。 此函数以我们先前定义的哈密顿量、振幅的界、门的持续时间、最大迭代次数和目标失真度。通过调用 `remoteOptimizeCr()`，用户可以向量脉云服务提交优化任务。如果用户想要进一步减少失真度，我们建议用户尝试增加门时间 `tg`（CR 门的持续时间约为 200 到 400 纳秒）。用户还可以通过设置更大的 `aBound` 和 `maxIter` 来增大搜索空间。

在本教程中，用来评估生成的量子门性能的失真度为 ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$，其中 $U_{\rm goal}$ 是目标酉变换 $U_{\rm CR}(-\pi /2)$，$d$ 是 $U_{\rm goal}$ 的维度，$U$ 是前面定义的三级体系的实际酉演化。注意，$P(U)$ 描述投影到计算子空间的演化。

In [ ]:
# Set amplitude bound
aBound = (-1.0, 3.0)

# Run the optimization
gateJob, infidelity = remoteOptimizeCr(ham, aBound=aBound, tg=200, maxIter=5, targetInfidelity=0.005)

我们可以使用 `plot()` 可视化生成的脉冲。有关 `plot()` 的详细信息，请参考 [single-qubit-gate](https://quanlse.baidu.com/#/doc/tutorial-single-qubit)。

In [ ]:
# Print waves and the infidelity
gateJob.plot()
print(f'infidelity: {infidelity}')

使用函数 `simulate()` 以及 `project()`，用户可以获得系统投影到计算子空间的演化后的矩阵 $P(U)$。

In [ ]:
# Print the system's evolution
result = ham.simulate(job=gateJob)
process2d = project(result[0]["unitary"], qubits, level, 2)
print("The projected evolution P(U):\n", round(process2d, 2))

此外，如果想要获取生成脉冲的每个时刻 `dt` 的数值，可以使用函数 `getPulseSequences()`。该函数采用哈密顿的字典和脉冲的通道名称作为输入参数。

In [ ]:
gateJob.generatePulseSequence(driveX(3), 0)
gateJob.plot()

## 总结


通过构造系统哈密顿量并在量脉云服务上产生优化的脉冲，我们成功地设计了一个可以实现高保真度的 cross-resonance 门的脉冲。我们鼓励用户尝试不同于本教程的参数值以获得最佳结果。

## 参考文献
 
\[1\] [Rigetti, Chad, and Michel Devoret. "Fully microwave-tunable universal gates in superconducting qubits with linear couplings and fixed transition frequencies." *Physical Review B* 81.13 (2010): 134507.](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.134507)

\[2\] [Nielsen, Michael A., and Isaac L. Chuang. Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge University Press, 2010.](http://csis.pace.edu/~ctappert/cs837-19spring/QC-textbook.pdf)